In [1]:
from newspaperedited import Article
import numpy as np  
import newspaper

## get urls and keywords from file

In [2]:
with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url.replace('\n', '') for url in urls]
urls

['https://vnexpress.net/', 'https://www.24h.com.vn/', 'https://tuoitre.vn/']

In [3]:
with open('keywords.txt', 'r') as f:
    keywords = f.readlines()
keywords = [key.replace('\n', '') for key in keywords]
keywords

['thu tuong', 'chinh phu', 'Viet Nam', 'pham minh chinh', 'nha nuoc']

### standardize keywords

In [4]:
keywords = [key.replace(' ', '_') for key in keywords]
keywords = [key.lower() for key in keywords]
list_key_tmp = []
for key in keywords:
    list_key_tmp += key.split('_')

# keywords += list_key_tmp
keywords

['thu_tuong', 'chinh_phu', 'viet_nam', 'pham_minh_chinh', 'nha_nuoc']

## crawl function

In [5]:
from bs4 import BeautifulSoup
import requests
import re  

def is_valid(url):
    # return re.findall(r'\.[a-z]{3}', url)
    return re.findall(r'\.[a-z]{3}', url) and re.search(r'https://', url)

# global dict
dict_url = {}

def get_link_articles_from_url(url):
    respone = requests.get(url)
    soup = BeautifulSoup(respone.text, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        articles_link = a['href'].replace('#box_comment_vne', '')
        if articles_link not in dict_url.keys() and is_valid(articles_link):
            dict_url[articles_link] = 1

    return list(dict_url.keys())

# test 
tmp = get_link_articles_from_url(urls[0])
print(len(tmp))
print(*tmp[:5], sep='\n')


27
https://e.vnexpress.net/
https://video.vnexpress.net
https://vnexpress.net/trinh-mien-nhiem-13-thanh-vien-chinh-phu-4259027.html
https://vnexpress.net/hien-truong-ho-tu-than-rong-hon-100-m2-4259284.html
https://vnexpress.net/bi-thu-dak-lak-bui-van-cuong-lam-tong-thu-ky-quoc-hoi-4259000.html


In [6]:
import newspaperedited

def get_category(url_home):
    dict_cate = {}
    home = newspaperedited.build(url_home)
    category_urls = home.category_urls()
    return list(set(category_urls))

list_cate = [get_category(url) for url in urls]

In [7]:
list_categorys = []
for i in list_cate:
    list_categorys += i  
list_categorys

['https://vnexpress.net/kinh-doanh',
 'https://vnexpress.net/the-gioi',
 'https://vnexpress.net/y-kien',
 'https://vnexpress.net/so-hoa',
 'https://vnexpress.net/the-thao',
 'https://vnexpress.net/tin-tuc-24h',
 'https://vnexpress.net/',
 'https://vnexpress.net/doi-song',
 'https://vnexpress.net/oto-xe-may',
 'https://e.vnexpress.net',
 'https://vnexpress.net/tam-su',
 'https://vnexpress.net/du-lich',
 'https://vnexpress.net/khoa-hoc',
 'https://vnexpress.net/thoi-su',
 'https://vnexpress.net/goc-nhin',
 'https://vnexpress.net/giai-tri',
 'https://vnexpress.net/giao-duc',
 'https://vnexpress.net/suc-khoe',
 'https://vnexpress.net/phap-luat',
 'https://vnexpress.net/hai',
 'https://vnexpress.net',
 'https://video.vnexpress.net',
 'https://www.24h.com.vn',
 'https://www.24h.com.vn/',
 'http://baogia.24h.com.vn',
 'https://tuoitre.vn/the-thao.htm',
 'https://tuoitre.vn/giai-tri.htm',
 'https://tv.tuoitre.vn',
 'https://cuoituan.tuoitre.vn',
 'https://tuoitre.vn/phap-luat.htm',
 'https://c

## get keywords from url

In [8]:
def get_text(url):
    text = ''
    try:
        article = Article(url)
        article.download()
        article.parse()
    except:
        return text
    text = article.text.replace('\n', '.\n')
    return text


In [9]:
from pyvi import ViTokenizer
from collections import Counter

def get_keywords_from_text(text):
    tokens = ViTokenizer.tokenize(text)
    tokens = ViTokenizer.spacy_tokenize(tokens)[0]
    tokens = list(filter(lambda x: len(x)>1, tokens))
    counter_tokens = Counter(tokens)
    counter_tokens = dict(counter_tokens)
    counter_tokens = dict(sorted(counter_tokens.items(), key=lambda x:-x[1]))
    return counter_tokens


In [10]:
def get_important_score(dict_keywords):
    score = 0
    # print('------------begin----------------------------')
    # print(keys)
    # print('-------------end---------------------------')
    for key in dict_keywords.keys():
        for k in keywords:
            if k == key.lower():
                score += dict_keywords[key]
    return score


## ranking articles

In [11]:
list_article_url = []

dict_url = {}
for category in list_categorys:
    list_article_url = get_link_articles_from_url(category)

In [12]:
len(list_article_url)

1287

In [13]:

list_text = []
cnt = 0
for article_url in list_article_url:
    text = get_text(article_url)
    if text != '':
        list_text.append(text)
    else:
        continue

    if cnt%10==0:
        print(len(list_text), end='-')
    cnt += 1

1-11-21-31-41-51-61-71-81-91-101-111-121-131-141-151-161-171-181-191-201-211-221-231-241-251-261-271-281-291-301-311-321-331-341-351-361-371-381-391-401-411-421-431-441-451-461-471-481-491-501-511-521-531-541-551-561-571-581-591-601-611-621-631-641-651-661-671-681-691-701-711-721-731-741-751-761-771-781-791-801-811-821-831-841-851-861-871-881-891-901-911-921-931-941-951-961-971-981-991-1001-1011-1021-1031-1041-1051-1061-1071-1081-1091-1101-1111-1121-1131-1141-1151-1161-1171-1181-1191-1201-1211-1221-

In [14]:
print(len(list_text))

1229


In [15]:
dict_index_and_score = {i:0 for i in range(len(list_text))}
# dict_index_and_score {0:0, 1:0, 2:0, .....}

In [16]:
list_keys = []
cnt = 0
for text in list_text:
    
    keys = get_keywords_from_text(text)
    dict_index_and_score[cnt] = get_important_score(keys)
    if cnt%10==0:
        print(cnt, end='-')
    cnt += 1


0-10-20-30-40-50-60-70-80-90-100-110-120-130-140-150-160-170-180-190-200-210-220-230-240-250-260-270-280-290-300-310-320-330-340-350-360-370-380-390-400-410-420-430-440-450-460-470-480-490-500-510-520-530-540-550-560-570-580-590-600-610-620-630-640-650-660-670-680-690-700-710-720-730-740-750-760-770-780-790-800-810-820-830-840-850-860-870-880-890-900-910-920-930-940-950-960-970-980-990-1000-1010-1020-1030-1040-1050-1060-1070-1080-1090-1100-1110-1120-1130-1140-1150-1160-1170-1180-1190-1200-1210-1220-

In [17]:
list_index = dict(sorted(dict_index_and_score.items(), key=lambda x : -x[1]))

In [18]:
cnt = 0 
for i in list_index:
    cnt+=1 
    print(i, list_index[i])
    print(list_article_url[i])
    if cnt == 20:
        break

374 1
https://e.vnexpress.net/news/business/companies/indian-motorcycle-maker-royal-enfield-bids-goodbye-to-vietnam-4258016.html
375 1
https://e.vnexpress.net/news/business/companies/indian-motorcycle-maker-royal-enfield-bids-goodbye-to-vietnam-4258016.html#box_comment
392 1
https://e.vnexpress.net/photo/places/da-nangs-tourist-attractions-swim-in-a-sea-of-clouds-4258792.html
393 1
https://e.vnexpress.net/photo/places/da-nangs-tourist-attractions-swim-in-a-sea-of-clouds-4258792.html#box_comment
0 0
https://e.vnexpress.net/
1 0
https://video.vnexpress.net
2 0
https://startup.vnexpress.net/
3 0
https://vhome.vnexpress.net/?utm_source=VnExpress_Desktop&utm_medium=KinhDoanh_Menu&utm_campaign=MenuNext_Startup
4 0
https://vnexpress.net/nhung-noi-nhieu-ty-phu-nhat-the-gioi-4259142.html
5 0
https://vnexpress.net/lan-song-san-van-phong-cua-doanh-nghiep-cong-nghe-tang-manh-4259095.html
6 0
https://vnexpress.net/10-ty-phu-tre-nhat-the-gioi-4259071.html
7 0
https://vnexpress.net/nhung-nguoi-thanh-

In [19]:
# list_article_url